In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/notebook0b472c722f/__results__.html
/kaggle/input/notebook0b472c722f/__notebook__.ipynb
/kaggle/input/notebook0b472c722f/tokenizer.pickle
/kaggle/input/notebook0b472c722f/custom.css
/kaggle/input/notebook0b472c722f/model.h5
/kaggle/input/notebook0b472c722f/__output__.json


In [2]:
import pickle
text1 = "I hated this product"
text2 = "It's a good one"
tokenizer = ''
with open('/kaggle/input/notebook0b472c722f/tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)
print(type(tokenizer))

<class 'keras_preprocessing.text.Tokenizer'>


In [3]:
from keras.models import load_model
model = load_model('/kaggle/input/notebook0b472c722f/model.h5')

In [4]:
import keras
print(keras.__version__)

2.4.3


In [5]:
from keras.preprocessing.sequence import pad_sequences
def predict_sentiment_of_the_review(review):
    encoded_doc_test = tokenizer.texts_to_sequences([review])
    padded_text_test=pad_sequences(encoded_doc_test,maxlen=4000, padding="post", truncating="post").astype('int16')
    res = model.predict(padded_text_test)
    print(res[0][0])
    if res[0][0]<0.5:
        print("It's a negative review")
        ans = False
    else:
        print("It's a positive review")
        ans = True
    return ans

In [6]:
print(text1)
predict_sentiment_of_the_review(text1)

I hated this product
0.019594759
It's a negative review


False

In [7]:
print(text2)
predict_sentiment_of_the_review(text2)

It's a good one
0.9790744
It's a positive review


True

In [8]:
text3 = "It's a bad one, but I loved it"
print(text3)
predict_sentiment_of_the_review(text3)

It's a bad one, but I loved it
0.8909942
It's a positive review


True

In [19]:
text3 = "I thought the product was bad, but it was decent"
print(text3)
predict_sentiment_of_the_review(text3)

I thought the product was bad, but it was decent
0.9879832
It's a positive review


True

In [10]:
print(model.layers[2].output.shape)

(None, 4000, 100)


In [11]:
from keras.models import Model
model_inter = Model(inputs=model.input, outputs=model.layers[2].output)


In [12]:
import re
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

stopwords= ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"]


from tqdm import tqdm
def preprocess_text(text_data):
    preprocessed_text = []
    for sentance in tqdm(text_data):
        sent = decontracted(sentance)
        sent = sent.replace('\\r', ' ')
        sent = sent.replace('\\n', ' ')
        sent = sent.replace('\\"', ' ')
        sent = re.sub('[^A-Za-z0-9]+', ' ', sent)
        #sent = ' '.join(e for e in sent.split() if e.lower() not in stopwords)
        preprocessed_text.append(sent.lower().strip())
    return preprocessed_text

In [13]:
text3 = "It's a good product, but I hated it"
text3 = preprocess_text([text3])
print(text3)
encoded_doc_test = tokenizer.texts_to_sequences([text3])
padded_text_test=pad_sequences(encoded_doc_test,maxlen=4000, padding="post", truncating="post").astype('int16')
inter_out = model_inter.predict(padded_text_test)

100%|██████████| 1/1 [00:00<00:00, 740.91it/s]

['it is a good product but i hated it']


In [ ]:
print(inter_out[0][6])
for i in range(4000):
    if not all(tf.equal(inter_out[0][i], zero)):
        print(i)

In [ ]:
good = inter_out[0][30]
hated = inter_out[0][2425]

In [ ]:
from numpy.linalg import norm
print(norm(good-hated))

In [ ]:
tokenizer.word_index['hated']

In [ ]:
import tensorflow as tf
zero = tf.constant(0, dtype=tf.float32)
where = tf.equal(inter_out, zero)

In [ ]:
where[0]